In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import savetxt
#from load_data import LoadData

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ![image.png](attachment:4d6d993c-ba0d-4eda-a63e-b3d6d06bacd2.png)

In [ ]:
!git clone https://github.com/SynergisticDrugCombinationPrediction/ConsDeepSignaling.git

In [ ]:
!pwd

In [2]:
# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Change the working directory
os.chdir('/home/hb/python/2022Lab/code/Lab18_Neural_Network/ConsDeepSignaling')

# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /home/hb/python/2022Lab/code/Lab18_Neural_Network
Current working directory: /home/hb/python/2022Lab/code/Lab18_Neural_Network/ConsDeepSignaling


In [3]:
cwd

'/home/hb/python/2022Lab/code/Lab18_Neural_Network'

In [4]:
# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Change the working directory
os.chdir('/content/drive/MyDrive/2021 생물정보학실습/4. Neural Network/ConsDeepSignaling')

# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /content
Current working directory: /content/drive/MyDrive/2021 생물정보학실습/4. Neural Network/ConsDeepSignaling


# Data parsing

In [160]:
from load_data import LoadData

class ParseFile():
    def __init__(self):
        pass

    # FIND THE DUPLICATE ROWS[CELL_LINE_NAME, DRUG_NAME, AUC] THEN AVERAGE SCORE
    def second_input_condense():
        '''FIND THE DUPLICATE ROWS[CELL_LINE_NAME, DRUG_NAME, AUC] THEN AVERAGE SCORE OF AUC BASED ON 'CELL_LINE_NAME' & 'DRUG_NAME'
        1. GDSC2_fitted_dose_response_25Feb20.xlsx 파일을 dl_second_df란 변수로 불러온다.
        2. dl_second_df에서 'CELL_LINE_NAME', 'DRUG_NAME', 'AUC' 이름의 열만 추출한다.
        3. 'CELL_LINE_NAME'과 'DRUG_NAME'을 기준으로 'AUC' values를 평균내어 집계한다. 
        4. 최종 데이터프레임을 ./datainfo/mid_data/GDSC2_dl_input.txt 파일로 저장한다. 
        '''
        print('\nREADING THE EXCEL FILE FOR DEEP LEARNING DOSE RESPONSE...')
        dl_second_df = pd.read_excel('./GDSC/dose_responce_25Feb20/GDSC2_fitted_dose_response_25Feb20.xlsx')
        dl_second_input_df = dl_second_df[['CELL_LINE_NAME', 'DRUG_NAME', 'AUC']]
        dl_second_input_df = dl_second_input_df.groupby(['CELL_LINE_NAME', 'DRUG_NAME']).agg({'AUC':'mean'}).reset_index()
        if os.path.exists('./datainfo/mid_data') == False:
            os.mkdir('./datainfo/mid_data')
        dl_second_input_df.to_csv('./datainfo/mid_data/GDSC2_dl_input.txt', index = False, header = True)
        print('--- DL INITIAL AVERAGE POINTS: ' + str(dl_second_input_df.shape[0]) + ' ---')

    # REMOVE INPUT ROWS WITH NO MAPPED DRUG NAME (FINALLY 39066 POINTS INPUT)
    def input_drug_condense():
        dl_input_df = pd.read_table('./datainfo/mid_data/GDSC2_dl_input.txt', delimiter = ',')
        drug_map_dict = ParseFile.drug_map_dict()
        deletion_list = []
        for row in dl_input_df.itertuples():
            if pd.isna(drug_map_dict[row[2]]):
                deletion_list.append(row[0])
        mid_dl_input_df = dl_input_df.drop(dl_input_df.index[deletion_list]).reset_index(drop = True)
        mid_dl_input_df.to_csv('./datainfo/mid_data/mid_GDSC2_dl_input.txt', index = False, header = True)
        print('--- DL DRUG CONDENSED INPUT POINTS: ' + str(mid_dl_input_df.shape[0]) + ' ---')

    # REMOVE INPUT ROWS WITH ALL ZEROS ON DRUG TARGET GENE CONNECTION, [FINAL 16761 POINTS]
    def input_drug_gene_condense():
        dir_opt = '/datainfo'
        deletion_list = []
        final_dl_input_df = pd.read_table('./datainfo/mid_data/final_GDSC2_dl_input.txt', delimiter = ',')
        drug_map_dict, drug_dict, gene_target_num_dict = LoadData(dir_opt).pre_load_dict()
        target_index_list = gene_target_num_dict.values()
        drug_target_matrix = np.load('./datainfo/filtered_data/drug_target_matrix.npy')
        for row in final_dl_input_df.itertuples():
            drug_a = drug_map_dict[row[2]]
            cellline_name = row[1]
            # DRUG_A AND 929 TARGET GENES
            drug_a_target_list = []
            drug_index = drug_dict[drug_a]
            for target_index in target_index_list:
                if target_index == -1 : 
                    effect = 0
                else:
                    effect = drug_target_matrix[drug_index, target_index]
                drug_a_target_list.append(effect)
            if all([a == 0 for a in drug_a_target_list]): 
                deletion_list.append(row[0])
        print('=====================' + str(len(deletion_list)))
        zero_final_dl_input_df = final_dl_input_df.drop(final_dl_input_df.index[deletion_list]).reset_index(drop = True)
        zero_final_dl_input_df.to_csv('.' + dir_opt + '/filtered_data/zerofinal_GDSC2_dl_input.txt', index = False, header = True)
        print(zero_final_dl_input_df)

    # RANDOMIZE THE DL INPUT
    def input_random_condense():
        zero_final_dl_input_df = pd.read_table('./datainfo/filtered_data/zerofinal_GDSC2_dl_input.txt', delimiter = ',')
        random_final_dl_input_df = zero_final_dl_input_df.sample(frac = 1)
        random_final_dl_input_df.to_csv('./datainfo/filtered_data/Randomfinal_GDSC2_dl_input.txt', index = False, header = True)
        print(random_final_dl_input_df)

    # SPLIT DEEP LEARNING INPUT INTO TRAINING AND TEST
    def split_k_fold(k, place_num):
        random_final_dl_input_df = pd.read_table('./datainfo/filtered_data/Randomfinal_GDSC2_dl_input.txt', delimiter = ',')
        print(random_final_dl_input_df)
        num_points = random_final_dl_input_df.shape[0]

        num_div = int(num_points / k)
        num_div_list = [i * num_div for i in range(0, k)]
        num_div_list.append(num_points)
        low_idx = num_div_list[place_num - 1]
        high_idx = num_div_list[place_num]
        print('\n--------TRAIN-TEST SPLIT WITH TEST FROM ' + str(low_idx) + ' TO ' + str(high_idx) + '--------')
        train_input_df = random_final_dl_input_df.drop(random_final_dl_input_df.index[low_idx : high_idx])
        print(train_input_df)
        test_input_df = random_final_dl_input_df[low_idx : high_idx]
        print(test_input_df)
        train_input_df.to_csv('./datainfo/filtered_data/TrainingInput.txt', index = False, header = True)
        test_input_df.to_csv('./datainfo/filtered_data/TestInput.txt', index = False, header = True)


    #########################################################

    # FORM DRUGS MAP BETWEEN [GDSC2_dl_input / drug_tar_bank]
    def drug_map():
        '''FROM DRUGS MAP BETWEEN [GDSC2_dl_input / drug_tar_bank]
        1. './datainfo/mid_data/GDSC2_dl_input.txt' 파일을 불러온다.
        2. './datainfo/init_data/drug_tar_drugBank_all.txt' 파일을 불러온다 : Drug이 표적하는 Target gene정보
        3. dl_input_df 있는 모든 DRUG_NAME 값 중 고유한값만 따로 GDSC2_input_drug_name.txt 파일로 저장해준다.
            => drug_df
        4. drug_target_df 있는 모든 Drug 값 중 고유한값만 따로 mapped_drug_name.txt 파일로 저장해준다. 
            => mapped_drug_df 
        5. drug_map_df란 변수에 drug_df와 mapped_drug_df를 병합해준 후 drug_map.csv 파일로 저장해준다. 
        AFTER AUTO MAP -> MANUAL MAP 
        '''
        dl_input_df = pd.read_table('./datainfo/mid_data/GDSC2_dl_input.txt', delimiter = ',')
        drug_target_df = pd.read_table('./datainfo/init_data/drug_tar_drugBank_all.txt')
        drug_list = []
        for drug in dl_input_df['DRUG_NAME']:
            if drug not in drug_list:
                drug_list.append(drug)
        drug_list = sorted(drug_list)
        drug_df = pd.DataFrame(data = drug_list, columns = ['Drug Name'])
        drug_df.to_csv('./datainfo/init_data/GDSC2_input_drug_name.txt', index = False, header = True)
        mapped_drug_list = []
        for drug in drug_target_df['Drug']:
            if drug not in mapped_drug_list:
                mapped_drug_list.append(drug)
        mapped_drug_list = sorted(mapped_drug_list)
        mapped_drug_df = pd.DataFrame(data = mapped_drug_list, columns = ['Mapped Drug Name'])
        mapped_drug_df.to_csv('./datainfo/init_data/mapped_drug_name.txt', index = False, header = True)
        # LEFT JOIN TWO DATAFRAME
        drug_map_df = pd.merge(drug_df, mapped_drug_df, how='left', left_on = 'Drug Name', right_on = 'Mapped Drug Name')
        drug_map_df.to_csv('./datainfo/init_data/drug_map.csv', index = False, header = True)
        # AFTER AUTO MAP -> MANUAL MAP
    
    # FROM MANUAL MAP TO DRUG MAP DICT
    def drug_map_dict():
        drug_map_df = pd.read_csv('./datainfo/mid_data/drug_map.csv')
        drug_map_dict = {}
        for row in drug_map_df.itertuples():
            drug_map_dict[row[1]] = row[2]
        if os.path.exists('./datainfo/filtered_data') == False:
            os.mkdir('./datainfo/filtered_data')
        np.save('./datainfo/filtered_data/drug_map_dict.npy', drug_map_dict)
        return drug_map_dict

    # FORM ADAJACENT MATRIX (DRUG x TARGET) (LIST -> SORTED -> DICT -> MATRIX) (ALL 5435 DRUGS <-> ALL 2775 GENES)
    def drug_target():
        drug_target_df = pd.read_table('./datainfo/init_data/drug_tar_drugBank_all.txt')
        # GET UNIQUE SORTED DRUGLIST AND TARGET(GENE) LIST
        drug_list = []
        for drug in drug_target_df['Drug']:
            if drug not in drug_list:
                drug_list.append(drug)
        drug_list = sorted(drug_list)
        target_list = []
        for target in drug_target_df['Target']:
            if target not in target_list:
                target_list.append(target)
        target_list = sorted(target_list)
        # CONVERT THE SORTED LIST TO DICT WITH VALUE OF INDEX
        drug_dict = {drug_list[i] : i for i in range((len(drug_list)))} 
        drug_num_dict = {i : drug_list[i] for i in range((len(drug_list)))} 
        target_dict = {target_list[i] : i for i in range(len(target_list))}
        target_num_dict = {i : target_list[i] for i in range(len(target_list))}
        # ITERATE THE DATAFRAME TO DEFINE CONNETIONS BETWEEN DRUG AND TARGET(GENE)
        drug_target_matrix = np.zeros((len(drug_list), len(target_list))).astype(int)
        for index, drug_target in drug_target_df.iterrows():
            # BUILD ADJACENT MATRIX
            drug_target_matrix[drug_dict[drug_target['Drug']], target_dict[drug_target['Target']]] = 1
        drug_target_matrix = drug_target_matrix.astype(int)
        np.save('./datainfo/filtered_data/drug_target_matrix.npy', drug_target_matrix)
        # np.savetxt("drug_target_matrix.csv", drug_target_matrix, delimiter=',')
        # x, y = drug_target_matrix.shape
        # for i in range(x):
        #     # FIND DRUG TARGET OVER 100 GENES
        #     row = drug_target_matrix[i, :]
        #     if len(row[row>=1]) >= 100: print(drug_num_dict[i])
        np.save('./datainfo/filtered_data/drug_dict.npy', drug_dict)
        np.save('./datainfo/filtered_data/drug_num_dict.npy', drug_num_dict)
        np.save('./datainfo/filtered_data/target_dict.npy', target_dict)
        np.save('./datainfo/filtered_data/target_num_dict.npy', target_num_dict)
        return drug_dict, drug_num_dict, target_dict, target_num_dict

    # FILTER [RNA_Seq / CopyNumber] SPARSE GENES
    def rna_cpnum_filter(rna_filter, cpnum_filter):
        print('\nFILTERING SPARSE FEATURE GENES OF RNA_Seq & CpNum...')
        rna_df = pd.read_csv('./GDSC/rnaseq_20191101/rnaseq_fpkm_20191101.csv', low_memory = False)
        # rna_df = rna_df.fillna(0.0)
        cpnum_df = pd.read_csv('./GDSC/cnv_20191101/cnv_gistic_20191101.csv', low_memory = False)
        # RNA_Seq FILTER GENES
        if rna_filter == True:
            print(rna_df.shape)
            rna_df = rna_df.drop_duplicates(subset = ['symbol'], 
                        keep = 'first').sort_values(by = ['symbol']).reset_index(drop = True)
            threshold = int((len(rna_df.columns) - 3) / 3)
            deletion_list = []
            for row in rna_df.itertuples():
                if list(row[3:]).count(0) > threshold: 
                    deletion_list.append(row[0])
            rna_df = rna_df.drop(rna_df.index[deletion_list]).reset_index(drop = True)
            rna_df.to_csv('./GDSC/rnaseq_20191101/filtered_rnaseq_fpkm_20191101.csv', index = False, header = True)
            print(rna_df.shape)
        # CopyNumber FILTER GENES
        if cpnum_filter == True:
            print(cpnum_df.shape)
            cpnum_df = cpnum_df.drop_duplicates(subset = ['symbol'], 
                        keep = 'first').sort_values(by = ['symbol']).reset_index(drop = True)
            threshold = int((len(cpnum_df.columns) - 3) / 3)
            deletion_list = []
            for row in cpnum_df.itertuples():
                if list(row[3:]).count(0) > threshold: 
                    deletion_list.append(row[0])
            cpnum_df = cpnum_df.drop(cpnum_df.index[deletion_list]).reset_index(drop = True)
            cpnum_df.to_csv('./GDSC/cnv_20191101/filtered_cnv_gistic_20191101.csv', index = False, header = True)
            print(cpnum_df.shape)


    # GET [RNA_Seq / CopyNumber] CELL LINES NAMES & GENES, FINALLY [9268  / 971 CELLLINES]
    def rna_cpnum_intersect(rna_filter, cpnum_filter):
        print('\nFINDING INTERSECTION OF RNA_Seq & CpNum...')
        if rna_filter == True:
            rna_df = pd.read_csv('./GDSC/rnaseq_20191101/filtered_rnaseq_fpkm_20191101.csv', low_memory = False)
        else:
            rna_df = pd.read_csv('./GDSC/rnaseq_20191101/rnaseq_fpkm_20191101.csv', low_memory = False)
        rna_cellline_list = list(rna_df.columns)
        rna_gene_list = list(rna_df['symbol'])
        if cpnum_filter == True:
            cpnum_df = pd.read_csv('./GDSC/cnv_20191101/filtered_cnv_gistic_20191101.csv', low_memory = False)
        else:
            cpnum_df = pd.read_csv('./GDSC/cnv_20191101/cnv_gistic_20191101.csv', low_memory = False)
        cpnum_cellline_list = list(cpnum_df.columns)
        cpnum_gene_list = list(cpnum_df['symbol']) 
        # GET INTERSECTION OF [RNA_Seq / CopyNumber]
        rna_cellline_set = set(rna_cellline_list)
        cpnum_cellline_set = set(cpnum_cellline_list)
        common_cellline_list = list(rna_cellline_set.intersection(cpnum_cellline_set))
        rna_gene_set = set(rna_gene_list)
        cpnum_gene_set = set(cpnum_gene_list)
        common_gene_list = list(rna_gene_set.intersection(cpnum_gene_set))
        # DELETE [Cell Lines / Genes] NOT IN RNA_Seq
        rna_cellline_deletion_list = []
        for cellline in rna_cellline_list:
            if cellline not in common_cellline_list:
                rna_cellline_deletion_list.append(cellline)
        tail_cellline_rna_df = rna_df.drop(columns = rna_cellline_deletion_list)
        rna_gene_deletion_list = []
        for gene in rna_gene_list:
            if gene not in common_gene_list:
                rna_gene_deletion_list.append(gene)
        rna_gene_deletion_index = []
        for row in tail_cellline_rna_df.itertuples():
            if row[2] in rna_gene_deletion_list:
                rna_gene_deletion_index.append(row[0])
        tailed_rna_df = tail_cellline_rna_df.drop(rna_gene_deletion_index).reset_index(drop = True)
        tailed_sort_rna_df = tailed_rna_df.sort_values(by = ['symbol'])
        tailed_sort_rna_df.to_csv('./GDSC/rnaseq_20191101/tailed_rnaseq_fpkm_20191101.csv', index = False, header = True)
        # print(tailed_sort_rna_df)
        # DELETE [Cell Lines / Genes] NOT IN CopyNumber
        cpnum_cellline_deletion_list = []
        for cellline in cpnum_cellline_list:
            if cellline not in common_cellline_list:
                cpnum_cellline_deletion_list.append(cellline)
        tail_cellline_cpnum_df = cpnum_df.drop(columns = cpnum_cellline_deletion_list)
        cpnum_gene_deletion_list = []
        for gene in cpnum_gene_list:
            if gene not in common_gene_list:
                cpnum_gene_deletion_list.append(gene)
        cpnum_gene_deletion_index = []
        for row in tail_cellline_cpnum_df.itertuples():
            if row[2] in cpnum_gene_deletion_list:
                cpnum_gene_deletion_index.append(row[0])
        tailed_cpnum_df = tail_cellline_cpnum_df.drop(cpnum_gene_deletion_index).reset_index(drop = True)
        tailed_sort_cpnum_df = tailed_cpnum_df.sort_values(by = ['symbol'])
        tailed_sort_cpnum_df.to_csv('./GDSC/cnv_20191101/tailed_cnv_gistic_20191101.csv', index = False, header = True)
        # print(tailed_sort_cpnum_df)
        # CONFIRMATION ON TAILED FILES' [GENES, CELLLINES] ORDER
        tailed_rna_cellline_list = list(tailed_sort_rna_df.columns)
        tailed_rna_gene_list = list(tailed_sort_rna_df['symbol'])
        tailed_cpnum_cellline_list = list(tailed_sort_cpnum_df.columns)
        tailed_cpnum_gene_list = list(tailed_sort_cpnum_df['symbol']) 
        error = 0
        for (rna_cl, cpnum_cl) in zip(tailed_rna_cellline_list, tailed_cpnum_cellline_list):
            if rna_cl != cpnum_cl: error = 1
        for (rna_gene, cpnum_gene) in zip(tailed_rna_gene_list, tailed_cpnum_gene_list):
            if rna_gene != cpnum_gene: error = 2
        if error == 0: 
            print('--- CONFIRMED ON IDENTICAL OF [RNA_Seq, CpNum] ---')
        print('--- GDSC(RNA_Seq, CpNum) INTERSECTION [GENES, CELLLINES]: ' + str(tailed_sort_cpnum_df.shape) + ' ---')


    # FIND CELLLINES INTERSECTION BETWEEN [GDSC2_dl_input / CpNum, RNA_Seq], THEN CONDENSE [mid_GDSC2_dl_input]
    # (FINALLY 38227 POINTS INPUT)
    def cellline_intersect_input_condense():
        # CELLLINES IN [mid_GDSC2_dl_input]
        mid_dl_input_df = pd.read_table('./datainfo/mid_data/mid_GDSC2_dl_input.txt', delimiter = ',')
        input_cellline_name = list(mid_dl_input_df['CELL_LINE_NAME'])
        input_cellline_name_list = []
        for cellline in input_cellline_name:
            if cellline not in input_cellline_name_list:
                input_cellline_name_list.append(cellline)
        # CELLLINES IN [RNA_Seq, CpNum]
        rna_df = pd.read_csv('./GDSC/rnaseq_20191101/tailed_rnaseq_fpkm_20191101.csv')
        rna_cellline_list = list(rna_df.columns)
        rna_cellline_list.remove('gene_id')
        rna_cellline_list.remove('symbol')
        cpnum_df = pd.read_csv('./GDSC/cnv_20191101/tailed_cnv_gistic_20191101.csv')
        cpnum_cellline_list = list(cpnum_df.columns)
        cpnum_cellline_list.remove('gene_id')
        cpnum_cellline_list.remove('symbol')
        # GET INTERSECTION OF [GDSC2_dl_input, RNA_Seq]
        rna_cellline_set = set(rna_cellline_list)
        input_cellline_set = set(input_cellline_name_list)
        common_cellline_list = list(rna_cellline_set.intersection(input_cellline_set))
        # REMOVE CELLLINES FOR [CpNum, RNA_Seq]
        print('\n[DL INPUT] REMOVING RNA-Seq/CpNum OUTER CELL LINES ...')
        cellline_deletion_list = []
        for cellline in rna_cellline_list:
            if cellline not in common_cellline_list:
                cellline_deletion_list.append(cellline)
        tail_cellline_rna_df = rna_df.drop(columns = cellline_deletion_list)
        tail_cellline_rna_df.to_csv('./datainfo/mid_data/intersect_rnaseq_fpkm_20191101.csv', index = False, header = True)
        tail_cellline_cpnum_df = cpnum_df.drop(columns = cellline_deletion_list)
        tail_cellline_cpnum_df.to_csv('./datainfo/mid_data/intersect_cnv_gistic_20191101.csv', index = False, header = True)
        print('--- GDSC(RNA_Seq, CpNum, mid_dl_input) INTERSECTION [GENES, CELLLINES]: ' + str(tail_cellline_cpnum_df.shape) + '---')
        # REMOVE CELLLINES FOR [mid_GDSC2_dl_input]
        print('[DL INPUT] REMOVING mid_GDSC2_dl_input OUTER CELL LINES ...')
        mid_dl_input_deletion_list = []
        for row in mid_dl_input_df.itertuples():
            if row[1] not in common_cellline_list:
                mid_dl_input_deletion_list.append(row[0])
        final_dl_input_df = mid_dl_input_df.drop(mid_dl_input_df.index[mid_dl_input_deletion_list]).reset_index(drop = True)
        final_dl_input_df.to_csv('./datainfo/mid_data/final_GDSC2_dl_input.txt', index = False, header = True)
        print('--- DL FINAL INPUT POINTS: ' + str(final_dl_input_df.shape[0]) + ' ---')


    # FIND GENES INTERSECTION BETWEEN [Selected_Kegg_Pathways2 / CpNum, RNA_Seq], COMMON GENES [TT1954 -> TF929]
    def gene_intersect_gdsc_condense():
        gene_pathway_df = pd.read_table('./datainfo/init_data/Selected_Kegg_Pathways2.txt')
        gene_list = list(gene_pathway_df['AllGenes'])
        intersect_rna_df = pd.read_csv('./datainfo/mid_data/intersect_rnaseq_fpkm_20191101.csv')
        intersect_cpnum_df = pd.read_csv('./datainfo/mid_data/intersect_cnv_gistic_20191101.csv')
        rna_gene_list = list(intersect_rna_df['symbol'])
        rna_gene_set = set(rna_gene_list)
        pathway_gene_set = set(gene_list)
        common_gene_list = list(rna_gene_set.intersection(pathway_gene_set))
        print('\n[PATHWAY GENE] REMOVING RNA-Seq/CpNum OUTER GENES ...')
        gene_deletion_list = []
        for gene in rna_gene_list:
            if gene not in common_gene_list:
                gene_deletion_list.append(gene)
        rna_gene_deletion_index = []
        for row in intersect_rna_df.itertuples():
            if row[2] in gene_deletion_list:
                rna_gene_deletion_index.append(row[0])
        tailed_rna_df = intersect_rna_df.drop(rna_gene_deletion_index).reset_index(drop = True)
        tailed_rna_df = tailed_rna_df.fillna(0.0)
        tailed_rna_df.to_csv('./datainfo/filtered_data/tailed_rnaseq_fpkm_20191101.csv', index = False, header = True)
        # print(tailed_rna_df)
        cpnum_gene_deletion_index = []
        for row in intersect_cpnum_df.itertuples():
            if row[2] in gene_deletion_list:
                cpnum_gene_deletion_index.append(row[0])
        tailed_cpnum_df = intersect_cpnum_df.drop(cpnum_gene_deletion_index).reset_index(drop = True)
        tailed_cpnum_df.to_csv('./datainfo/filtered_data/tailed_cnv_gistic_20191101.csv', index = False, header = True)
        # print(tailed_cpnum_df)
        # print(tailed_rna_df.isnull().sum())
        print('--- GDSC(RNA_Seq, CpNum) FINAL [GENES, CELLLINES]: ' + str(tailed_cpnum_df.shape) + ' ---')


    # [GDSC RNA_Seq/CpNum GENES : DRUG_TAR GENES]  KEY : VALUE
    def gene_target_num_dict():
        drug_dict, drug_num_dict, target_dict, target_num_dict = ParseFile.drug_target()
        rna_df = pd.read_csv('./datainfo/filtered_data/tailed_rnaseq_fpkm_20191101.csv')
        print(rna_df.shape)
        # print(target_dict)
        gene_target_num_dict = {}
        count = 0
        num = 0
        for row in rna_df.itertuples():
            if row[2] not in target_dict.keys(): 
                map_index = -1
                count += 1
            else:
                map_index = target_dict[row[2]]
                num += 1
            gene_target_num_dict[row[0]] = map_index
        print(count)
        print(num)
        np.save('./datainfo/filtered_data/gene_target_num_dict.npy', gene_target_num_dict)
        return gene_target_num_dict
    

    # FORM ADAJACENT FILE
    # THEN CONVERT INTO MATRIX (GENE x PATHWAY) (LIST -> SORTED -> DICT -> MATRIX)
    def gene_pathway():
        gene_pathway_df = pd.read_table('./datainfo/init_data/Selected_Kegg_Pathways2.txt')
        gene_pathway_df = gene_pathway_df.sort_values(by = ['AllGenes']).reset_index(drop = True)
        gene_list = list(gene_pathway_df['AllGenes'])
        gene_pathway_df = gene_pathway_df.drop(['AllGenes'], axis = 1)
        pathway_list = list(gene_pathway_df.columns)
        # CONVERT SORTED LIST TO DICT WITH INDEX
        gene_dict = {gene_list[i] : i for i in range(len(gene_list))}
        gene_num_dict = {i : gene_list[i] for i in range(len(gene_list))}
        pathway_dict = {pathway_list[i] : i for i in range(len(pathway_list))}
        pathway_num_dict = {i : pathway_list[i] for i in range(len(pathway_list))}
        # ITERATE THE DATAFRAME TO DEFINE CONNECTIONS BETWEEN GENES AND PATHWAYS
        gene_pathway_matrix = np.zeros((len(gene_list), len(pathway_list))).astype(int)
        for gene_row in gene_pathway_df.itertuples():
            pathway_index = 0
            for gene in gene_row[1:]:
                if gene != 'test':
                    gene_pathway_matrix[gene_dict[gene], pathway_index] = 1.0
                pathway_index += 1
        np.save('./datainfo/mid_data/gene_pathway_matrix.npy', gene_pathway_matrix)
        order_gene_pathway_df = pd.DataFrame(data = gene_pathway_matrix,
            index = [gene for gene in gene_list],
            columns = [pathway for pathway in pathway_list])
        order_gene_pathway_df.to_csv('./datainfo/mid_data/Ordered_Selected_Kegg_Pathways2.csv', index = True, header = True)
        print(order_gene_pathway_df.shape)
        

    # PARSE THOSE GENES NOT IN [RNA_Seq / CpNum]
    def gene_pathway_parse():
        order_gene_pathway_df = pd.read_csv('./datainfo/mid_data/Ordered_Selected_Kegg_Pathways2.csv')
        rna_df = pd.read_csv('./datainfo/filtered_data/tailed_rnaseq_fpkm_20191101.csv')
        cpnum_df = pd.read_csv('./datainfo/filtered_data/tailed_cnv_gistic_20191101.csv')
        rna_gene_list = list(rna_df['symbol'])
        deletion_list = []
        for row in order_gene_pathway_df.itertuples():
            if row[1] not in rna_gene_list:
                deletion_list.append(row[0])
        tailed_gene_pathway_df = order_gene_pathway_df.drop(deletion_list).reset_index(drop = True)
        tailed_gene_pathway_df.to_csv('./datainfo/filtered_data/Tailed_Selected_Kegg_Pathways2.csv', index = False, header = True)
        tailed_gene_pathway_df = tailed_gene_pathway_df.drop(columns = ['Unnamed: 0'])
        print(tailed_gene_pathway_df.shape)
        # CONFIRMATION ON TAILED FILES' [pathway_gene, rna_seq]  GENES IDNETICAL
        tailed_rna_cellline_list = list(rna_df.columns)
        tailed_rna_gene_list = list(rna_df['symbol'])
        tailed_cpnum_cellline_list = list(cpnum_df.columns)
        tailed_cpnum_gene_list = list(cpnum_df['symbol']) 
        error = 0
        for (rna_cl, cpnum_cl) in zip(tailed_rna_cellline_list, tailed_cpnum_cellline_list):
            if rna_cl != cpnum_cl: error = 1
        for (rna_gene, cpnum_gene) in zip(tailed_rna_gene_list, tailed_cpnum_gene_list):
            if rna_gene != cpnum_gene: error = 2
        if error == 0: 
            print('--- CONFIRMED ON IDENTICAL OF [RNA_Seq, CpNum] ---')
        np.save('./datainfo/filtered_data/gene_pathway_matrix.npy', tailed_gene_pathway_df.values)
        

# FORM DRUG MAP BETWEEN [GDSC2_dl_input, drug_tar_drugBank]
def pre_drug_manual():
    '''FORM DRUG MAP BETWEEN [GDSC2_dl_input, drug_tar_drugBank] 
    1) run ParseFile.second_input_condense()
    2) run ParseFile.drug_map()

    AFTER GET [/init_data/drug_map.csv] WITH AUTO MAP -> MANUAL MAP
    '''
    ParseFile.second_input_condense()
    ParseFile.drug_map()
    # AFTER GET [/init_data/drug_map.csv] WITH AUTO MAP -> MANUAL MAP

# REMOVE DRUGS IN [GDSC2_dl_input] NOT IN [drug_tar_drugBank]
def pre_drug_parse():
    ParseFile.drug_map_dict()
    ParseFile.drug_target()
    ParseFile.input_drug_condense()

# INPUT CELLLINE CONDENSE BY INTERSECTION BETWEEN [mid_GDSC2_dl_input, CpNum, RNA_Seq]
def pre_cellline_parse():
    rna_filter = True
    cpnum_filter = False
    ParseFile.rna_cpnum_filter(rna_filter, cpnum_filter)
    ParseFile.rna_cpnum_intersect(rna_filter, cpnum_filter)
    ParseFile.cellline_intersect_input_condense()
    
# REMOVE GENES IN [CpNum / RNA_Seq] NOT IN [Selected_Kegg_Pathways2]
def pre_gene_parse():
    ParseFile.gene_intersect_gdsc_condense()

# BUILD DICTIONARY FOR
def pre_relation():
    ParseFile.gene_target_num_dict()
    ParseFile.gene_pathway()
    ParseFile.gene_pathway_parse()

# FINAL INPUT PARSE WITH ALL ZERO ON DRUG TARGET
def zero_final():
    ParseFile.input_drug_gene_condense()

def k_fold_split(random_mode, k, place_num):
    dir_opt = '/datainfo2'
    if random_mode == True:
        ParseFile.input_random_condense()
    ParseFile.split_k_fold(k, place_num)

In [ ]:
def drug_map():
    dl_input_df = pd.read_table('./datainfo/mid_data/GDSC2_dl_input.txt', delimiter = ',')
    drug_target_df = pd.read_table('./datainfo/init_data/drug_tar_drugBank_all.txt')
    drug_list = []
    for drug in dl_input_df['DRUG_NAME']:
        if drug not in drug_list:
            drug_list.append(drug)
    drug_list = sorted(drug_list)
    drug_df = pd.DataFrame(data = drug_list, columns = ['Drug Name'])
    drug_df.to_csv('./datainfo/init_data/GDSC2_input_drug_name.txt', index = False, header = True)
    mapped_drug_list = []
    for drug in drug_target_df['Drug']:
        if drug not in mapped_drug_list:
            mapped_drug_list.append(drug)
    mapped_drug_list = sorted(mapped_drug_list)
    mapped_drug_df = pd.DataFrame(data = mapped_drug_list, columns = ['Mapped Drug Name'])
    mapped_drug_df.to_csv('./datainfo/init_data/mapped_drug_name.txt', index = False, header = True)
    # LEFT JOIN TWO DATAFRAME
    drug_map_df = pd.merge(drug_df, mapped_drug_df, how='left', left_on = 'Drug Name', right_on = 'Mapped Drug Name')
    drug_map_df.to_csv('./datainfo/init_data/drug_map.csv', index = False, header = True)
    # AFTER AUTO MAP -> MANUAL MAP


In [171]:
print(pre_drug_manual.__doc__)
print('-'*100)
print(ParseFile.second_input_condense.__doc__)
print('-'*100)
print(ParseFile.drug_map.__doc__)

FORM DRUG MAP BETWEEN [GDSC2_dl_input, drug_tar_drugBank] 
    1) run ParseFile.second_input_condense()
    2) run ParseFile.drug_map()

    AFTER GET [/init_data/drug_map.csv] WITH AUTO MAP -> MANUAL MAP
    
----------------------------------------------------------------------------------------------------
FIND THE DUPLICATE ROWS[CELL_LINE_NAME, DRUG_NAME, AUC] THEN AVERAGE SCORE OF AUC BASED ON 'CELL_LINE_NAME' & 'DRUG_NAME'
        1. GDSC2_fitted_dose_response_25Feb20.xlsx 파일을 dl_second_df란 변수로 불러온다.
        2. dl_second_df에서 'CELL_LINE_NAME', 'DRUG_NAME', 'AUC' 이름의 열만 추출한다.
        3. 'CELL_LINE_NAME'과 'DRUG_NAME'을 기준으로 'AUC' values를 평균내어 집계한다. 
        4. 최종 데이터프레임을 ./datainfo/mid_data/GDSC2_dl_input.txt 파일로 저장한다. 
        
----------------------------------------------------------------------------------------------------
FROM DRUGS MAP BETWEEN [GDSC2_dl_input / drug_tar_bank]
        1. './datainfo/mid_data/GDSC2_dl_input.txt' 파일을 불러온다.
        2. './datainfo/init_data/drug_t

In [5]:
# # Manually Fix the Drug Map Problem
pre_drug_manual()


READING THE EXCEL FILE FOR DEEP LEARNING DOSE RESPONSE...
--- DL INITIAL AVERAGE POINTS: 131108 ---


In [6]:
pre_drug_parse()

--- DL DRUG CONDENSED INPUT POINTS: 39066 ---


In [7]:
def pre_cellline_parse():
    rna_filter = True
    cpnum_filter = False
    ParseFile.rna_cpnum_filter(rna_filter, cpnum_filter)
    ParseFile.rna_cpnum_intersect(rna_filter, cpnum_filter)
    ParseFile.cellline_intersect_input_condense()
    
# REMOVE GENES IN [CpNum / RNA_Seq] NOT IN [Selected_Kegg_Pathways2]
def pre_gene_parse():
    ParseFile.gene_intersect_gdsc_condense()

In [8]:
rna_df = pd.read_csv('./GDSC/rnaseq_20191101/rnaseq_fpkm_20191101.csv', low_memory = False)
# rna_df = rna_df.fillna(0.0)
cpnum_df = pd.read_csv('./GDSC/cnv_20191101/cnv_gistic_20191101.csv', low_memory = False)

In [9]:
rna_df.head()

,gene_id,symbol,MEC-1,NBsusSR,M14,MDA-MB-134-VI,MCC26,MCC13,MCAS,MC-1010,...,MMAc-SF,BE-13,MC-IXC,Ramos-2G6-4C10,CGTH-W-1,H9,P3HR-1,HUH-6-clone5,GR-ST,MM1S
0,SIDG00001,A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SIDG00002,A1BG-AS1,0.0,1.0,3.0,2.0,2.0,4.0,0.0,4.0,...,4.0,2.0,2.0,6.0,3.0,0.0,0.0,0.0,4.0,0.0
2,SIDG00003,A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
3,SIDG00004,A2M,0.0,0.0,8.0,0.0,23.0,0.0,0.0,0.0,...,8.0,0.0,1.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0
4,SIDG00005,A2M-AS1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
cpnum_df.head()

,gene_id,symbol,M14,TE-12,TMK-1,STS-0421,PL4,PCI-4B,PCI-30,HSC-39,...,451Lu,MMAc-SF,BE-13,MC-IXC,Ramos-2G6-4C10,CGTH-W-1,H9,GR-ST,YMB-1-E,MM1S
0,SIDG00001,A1BG,0,-1,0,0,1,-1,0,0,...,0,-1,2,0,0,1,1,0,0,0
1,SIDG00002,A1BG-AS1,0,-1,0,0,1,-1,0,0,...,0,-1,2,0,0,1,1,0,0,0
2,SIDG00003,A1CF,-1,0,0,1,-1,0,1,0,...,0,-1,-1,-1,0,0,1,0,0,0
3,SIDG00004,A2M,0,1,-1,0,1,1,1,-1,...,1,0,0,0,0,1,0,0,1,0
4,SIDG00006,A2ML1,0,1,-1,0,1,0,1,-1,...,1,0,0,0,0,1,0,0,1,0


In [11]:
rna_df = rna_df.drop_duplicates(subset = ['symbol'], 
                        keep = 'first').sort_values(by = ['symbol']).reset_index(drop = True)

In [12]:
rna_df

,gene_id,symbol,MEC-1,NBsusSR,M14,MDA-MB-134-VI,MCC26,MCC13,MCAS,MC-1010,...,MMAc-SF,BE-13,MC-IXC,Ramos-2G6-4C10,CGTH-W-1,H9,P3HR-1,HUH-6-clone5,GR-ST,MM1S
0,SIDG06271,1-Dec,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SIDG17276,1-Mar,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0
2,SIDG34079,1-Sep,20.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,...,0.0,13.0,0.0,23.0,0.0,24.0,20.0,0.0,29.0,22.0
3,SIDG17287,10-Mar,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SIDG34100,10-Sep,0.0,12.0,23.0,17.0,33.0,42.0,17.0,5.0,...,14.0,0.0,20.0,0.0,42.0,0.0,0.0,28.0,0.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37272,SIDG42471,ZYG11B,2.0,8.0,8.0,3.0,7.0,4.0,3.0,3.0,...,8.0,5.0,4.0,6.0,18.0,4.0,2.0,4.0,7.0,3.0
37273,SIDG42472,ZYX,8.0,15.0,51.0,6.0,141.0,78.0,40.0,25.0,...,27.0,4.0,46.0,0.0,54.0,44.0,1.0,19.0,8.0,11.0
37274,SIDG42473,ZYXP1,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
37275,SIDG42474,ZZEF1,3.0,6.0,7.0,3.0,3.0,4.0,2.0,6.0,...,7.0,5.0,2.0,3.0,6.0,18.0,9.0,2.0,20.0,8.0


In [13]:
threshold = int((len(rna_df.columns) - 3) / 3)
deletion_list = []
for row in rna_df.itertuples():
    if list(row[3:]).count(0) > threshold: 
        deletion_list.append(row[0])
rna_df = rna_df.drop(rna_df.index[deletion_list]).reset_index(drop = True)

In [14]:
rna_df

,gene_id,symbol,MEC-1,NBsusSR,M14,MDA-MB-134-VI,MCC26,MCC13,MCAS,MC-1010,...,MMAc-SF,BE-13,MC-IXC,Ramos-2G6-4C10,CGTH-W-1,H9,P3HR-1,HUH-6-clone5,GR-ST,MM1S
0,SIDG34100,10-Sep,0.0,12.0,23.0,17.0,33.0,42.0,17.0,5.0,...,14.0,0.0,20.0,0.0,42.0,0.0,0.0,28.0,0.0,26.0
1,SIDG34102,11-Sep,22.0,23.0,17.0,17.0,40.0,19.0,16.0,21.0,...,20.0,0.0,13.0,8.0,17.0,1.0,0.0,10.0,0.0,11.0
2,SIDG34080,2-Sep,57.0,48.0,50.0,63.0,106.0,77.0,53.0,46.0,...,52.0,50.0,62.0,73.0,42.0,37.0,63.0,46.0,52.0,25.0
3,SIDG17282,5-Mar,5.0,13.0,8.0,14.0,8.0,10.0,14.0,7.0,...,6.0,16.0,9.0,8.0,12.0,10.0,5.0,5.0,13.0,7.0
4,SIDG17283,6-Mar,17.0,15.0,10.0,49.0,27.0,13.0,24.0,22.0,...,25.0,47.0,25.0,26.0,18.0,16.0,10.0,17.0,16.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931,SIDG42468,ZXDC,2.0,15.0,4.0,7.0,4.0,5.0,3.0,3.0,...,3.0,8.0,11.0,3.0,7.0,10.0,3.0,8.0,14.0,6.0
9932,SIDG42471,ZYG11B,2.0,8.0,8.0,3.0,7.0,4.0,3.0,3.0,...,8.0,5.0,4.0,6.0,18.0,4.0,2.0,4.0,7.0,3.0
9933,SIDG42472,ZYX,8.0,15.0,51.0,6.0,141.0,78.0,40.0,25.0,...,27.0,4.0,46.0,0.0,54.0,44.0,1.0,19.0,8.0,11.0
9934,SIDG42474,ZZEF1,3.0,6.0,7.0,3.0,3.0,4.0,2.0,6.0,...,7.0,5.0,2.0,3.0,6.0,18.0,9.0,2.0,20.0,8.0


In [15]:
rna_df.to_csv('./GDSC/rnaseq_20191101/filtered_rnaseq_fpkm_20191101.csv', index = False, header = True)

In [16]:
rna_df = pd.read_csv('./GDSC/rnaseq_20191101/filtered_rnaseq_fpkm_20191101.csv', low_memory = False)

In [25]:
rna_df.columns

Index(['gene_id', 'symbol', 'MEC-1', 'NBsusSR', 'M14', 'MDA-MB-134-VI',
       'MCC26', 'MCC13', 'MCAS', 'MC-1010',
       ...
       'MMAc-SF', 'BE-13', 'MC-IXC', 'Ramos-2G6-4C10', 'CGTH-W-1', 'H9',
       'P3HR-1', 'HUH-6-clone5', 'GR-ST', 'MM1S'],
      dtype='object', length=1049)

In [27]:
rna_df['symbol']

0       10-Sep
1       11-Sep
2        2-Sep
3        5-Mar
4        6-Mar
         ...  
9931      ZXDC
9932    ZYG11B
9933       ZYX
9934     ZZEF1
9935      ZZZ3
Name: symbol, Length: 9936, dtype: object

In [37]:
def pre_gene_parse():
    ParseFile.gene_intersect_gdsc_condense()
rna_cellline_list = list(rna_df.columns)
rna_gene_list = list(rna_df['symbol'])

In [29]:
cpnum_df = pd.read_csv('./GDSC/cnv_20191101/cnv_gistic_20191101.csv', low_memory = False)

In [30]:
cpnum_df

,gene_id,symbol,M14,TE-12,TMK-1,STS-0421,PL4,PCI-4B,PCI-30,HSC-39,...,451Lu,MMAc-SF,BE-13,MC-IXC,Ramos-2G6-4C10,CGTH-W-1,H9,GR-ST,YMB-1-E,MM1S
0,SIDG00001,A1BG,0,-1,0,0,1,-1,0,0,...,0,-1,2,0,0,1,1,0,0,0
1,SIDG00002,A1BG-AS1,0,-1,0,0,1,-1,0,0,...,0,-1,2,0,0,1,1,0,0,0
2,SIDG00003,A1CF,-1,0,0,1,-1,0,1,0,...,0,-1,-1,-1,0,0,1,0,0,0
3,SIDG00004,A2M,0,1,-1,0,1,1,1,-1,...,1,0,0,0,0,1,0,0,1,0
4,SIDG00006,A2ML1,0,1,-1,0,1,0,1,-1,...,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20664,SIDG42469,ZYG11A,1,0,0,0,-1,0,1,0,...,1,0,0,0,-1,1,0,0,-1,0
20665,SIDG42471,ZYG11B,1,0,0,0,-1,0,1,0,...,1,0,0,0,-1,1,0,0,-1,0
20666,SIDG42472,ZYX,2,1,2,2,-1,-1,2,2,...,1,1,2,0,1,0,1,1,0,1
20667,SIDG42474,ZZEF1,2,0,-1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-2,1,1,0,-1,0


In [31]:
cpnum_cellline_list = list(cpnum_df.columns)
cpnum_gene_list = list(cpnum_df['symbol']) 

In [38]:
rna_cellline_set = set(rna_cellline_list)
cpnum_cellline_set = set(cpnum_cellline_list)
common_cellline_list = list(rna_cellline_set.intersection(cpnum_cellline_set))

In [34]:
rna_gene_set = set(rna_gene_list)
cpnum_gene_set = set(cpnum_gene_list)
common_gene_list = list(rna_gene_set.intersection(cpnum_gene_set))

In [39]:
rna_cellline_deletion_list = []
for cellline in rna_cellline_list:
    if cellline not in common_cellline_list:
        rna_cellline_deletion_list.append(cellline)
tail_cellline_rna_df = rna_df.drop(columns = rna_cellline_deletion_list)
rna_gene_deletion_list = []
for gene in rna_gene_list:
    if gene not in common_gene_list:
        rna_gene_deletion_list.append(gene)
rna_gene_deletion_index = []
for row in tail_cellline_rna_df.itertuples():
    if row[2] in rna_gene_deletion_list:
        rna_gene_deletion_index.append(row[0])
tailed_rna_df = tail_cellline_rna_df.drop(rna_gene_deletion_index).reset_index(drop = True)
tailed_sort_rna_df = tailed_rna_df.sort_values(by = ['symbol'])
tailed_sort_rna_df.to_csv('./GDSC/rnaseq_20191101/tailed_rnaseq_fpkm_20191101.csv', index = False, header = True)

In [40]:
rna_cellline_deletion_list = []
for cellline in rna_cellline_list:
    if cellline not in common_cellline_list:
        rna_cellline_deletion_list.append(cellline)
tail_cellline_rna_df = rna_df.drop(columns = rna_cellline_deletion_list)
rna_gene_deletion_list = []
for gene in rna_gene_list:
    if gene not in common_gene_list:
        rna_gene_deletion_list.append(gene)
rna_gene_deletion_index = []
for row in tail_cellline_rna_df.itertuples():
    if row[2] in rna_gene_deletion_list:
        rna_gene_deletion_index.append(row[0])
tailed_rna_df = tail_cellline_rna_df.drop(rna_gene_deletion_index).reset_index(drop = True)
tailed_sort_rna_df = tailed_rna_df.sort_values(by = ['symbol'])
tailed_sort_rna_df.to_csv('./GDSC/rnaseq_20191101/tailed_rnaseq_fpkm_20191101.csv', index = False, header = True)

In [41]:
cpnum_cellline_deletion_list = []
for cellline in cpnum_cellline_list:
    if cellline not in common_cellline_list:
        cpnum_cellline_deletion_list.append(cellline)
tail_cellline_cpnum_df = cpnum_df.drop(columns = cpnum_cellline_deletion_list)
cpnum_gene_deletion_list = []
for gene in cpnum_gene_list:
    if gene not in common_gene_list:
        cpnum_gene_deletion_list.append(gene)
cpnum_gene_deletion_index = []
for row in tail_cellline_cpnum_df.itertuples():
    if row[2] in cpnum_gene_deletion_list:
        cpnum_gene_deletion_index.append(row[0])
tailed_cpnum_df = tail_cellline_cpnum_df.drop(cpnum_gene_deletion_index).reset_index(drop = True)
tailed_sort_cpnum_df = tailed_cpnum_df.sort_values(by = ['symbol'])
tailed_sort_cpnum_df.to_csv('./GDSC/cnv_20191101/tailed_cnv_gistic_20191101.csv', index = False, header = True)
# print(tailed_sort_cpnum_df)
# CONFIRMATION ON TAILED FILES' [GENES, CELLLINES] ORDER
tailed_rna_cellline_list = list(tailed_sort_rna_df.columns)
tailed_rna_gene_list = list(tailed_sort_rna_df['symbol'])
tailed_cpnum_cellline_list = list(tailed_sort_cpnum_df.columns)
tailed_cpnum_gene_list = list(tailed_sort_cpnum_df['symbol']) 
error = 0
for (rna_cl, cpnum_cl) in zip(tailed_rna_cellline_list, tailed_cpnum_cellline_list):
    if rna_cl != cpnum_cl: error = 1
for (rna_gene, cpnum_gene) in zip(tailed_rna_gene_list, tailed_cpnum_gene_list):
    if rna_gene != cpnum_gene: error = 2
if error == 0: 
    print('--- CONFIRMED ON IDENTICAL OF [RNA_Seq, CpNum] ---')
print('--- GDSC(RNA_Seq, CpNum) INTERSECTION [GENES, CELLLINES]: ' + str(tailed_sort_cpnum_df.shape) + ' ---')

--- CONFIRMED ON IDENTICAL OF [RNA_Seq, CpNum] ---
--- GDSC(RNA_Seq, CpNum) INTERSECTION [GENES, CELLLINES]: (9268, 973) ---


In [42]:
mid_dl_input_df = pd.read_table('./datainfo/mid_data/mid_GDSC2_dl_input.txt', delimiter = ',')

In [43]:
mid_dl_input_df

,CELL_LINE_NAME,DRUG_NAME,AUC
0,22RV1,5-Fluorouracil,0.778340
1,22RV1,Afatinib,0.879934
2,22RV1,Alisertib,0.940927
3,22RV1,Bortezomib,0.779146
4,22RV1,Camptothecin,0.777766
...,...,...,...
39061,huH-1,Venetoclax,0.955071
39062,huH-1,Vinblastine,0.834914
39063,huH-1,Vincristine,0.683638
39064,huH-1,Vinorelbine,0.862141


In [44]:
input_cellline_name = list(mid_dl_input_df['CELL_LINE_NAME'])
input_cellline_name_list = []
for cellline in input_cellline_name:
    if cellline not in input_cellline_name_list:
        input_cellline_name_list.append(cellline)

In [45]:
rna_df = pd.read_csv('./GDSC/rnaseq_20191101/tailed_rnaseq_fpkm_20191101.csv')
rna_cellline_list = list(rna_df.columns)
rna_cellline_list.remove('gene_id')
rna_cellline_list.remove('symbol')
cpnum_df = pd.read_csv('./GDSC/cnv_20191101/tailed_cnv_gistic_20191101.csv')
cpnum_cellline_list = list(cpnum_df.columns)
cpnum_cellline_list.remove('gene_id')
cpnum_cellline_list.remove('symbol')

In [46]:
rna_df

,gene_id,symbol,M14,TE-12,TMK-1,STS-0421,PL4,PCI-4B,PCI-30,HSC-39,...,C3A,451Lu,MMAc-SF,BE-13,MC-IXC,Ramos-2G6-4C10,CGTH-W-1,H9,GR-ST,MM1S
0,SIDG34100,10-Sep,23.0,19.0,26.0,49.0,12.0,10.0,43.0,15.0,...,19.0,19.0,14.0,0.0,20.0,0.0,42.0,0.0,0.0,26.0
1,SIDG34102,11-Sep,17.0,10.0,1.0,8.0,15.0,7.0,19.0,3.0,...,5.0,18.0,20.0,0.0,13.0,8.0,17.0,1.0,0.0,11.0
2,SIDG34080,2-Sep,50.0,46.0,35.0,58.0,50.0,42.0,58.0,35.0,...,72.0,54.0,52.0,50.0,62.0,73.0,42.0,37.0,52.0,25.0
3,SIDG17282,5-Mar,8.0,10.0,12.0,14.0,7.0,12.0,12.0,14.0,...,14.0,6.0,6.0,16.0,9.0,8.0,12.0,10.0,13.0,7.0
4,SIDG17283,6-Mar,10.0,37.0,24.0,23.0,30.0,23.0,29.0,43.0,...,20.0,14.0,25.0,47.0,25.0,26.0,18.0,16.0,16.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9263,SIDG42468,ZXDC,4.0,4.0,1.0,4.0,8.0,7.0,5.0,11.0,...,5.0,8.0,3.0,8.0,11.0,3.0,7.0,10.0,14.0,6.0
9264,SIDG42471,ZYG11B,8.0,10.0,4.0,10.0,2.0,8.0,9.0,6.0,...,3.0,8.0,8.0,5.0,4.0,6.0,18.0,4.0,7.0,3.0
9265,SIDG42472,ZYX,51.0,21.0,26.0,136.0,12.0,25.0,97.0,29.0,...,10.0,50.0,27.0,4.0,46.0,0.0,54.0,44.0,8.0,11.0
9266,SIDG42474,ZZEF1,7.0,5.0,2.0,5.0,4.0,4.0,4.0,3.0,...,5.0,5.0,7.0,5.0,2.0,3.0,6.0,18.0,20.0,8.0


In [47]:
rna_cellline_set = set(rna_cellline_list)
input_cellline_set = set(input_cellline_name_list)
common_cellline_list = list(rna_cellline_set.intersection(input_cellline_set))
# REMOVE CELLLINES FOR [CpNum, RNA_Seq]
print('\n[DL INPUT] REMOVING RNA-Seq/CpNum OUTER CELL LINES ...')
cellline_deletion_list = []
for cellline in rna_cellline_list:
    if cellline not in common_cellline_list:
        cellline_deletion_list.append(cellline)
tail_cellline_rna_df = rna_df.drop(columns = cellline_deletion_list)
tail_cellline_rna_df.to_csv('./datainfo/mid_data/intersect_rnaseq_fpkm_20191101.csv', index = False, header = True)
tail_cellline_cpnum_df = cpnum_df.drop(columns = cellline_deletion_list)
tail_cellline_cpnum_df.to_csv('./datainfo/mid_data/intersect_cnv_gistic_20191101.csv', index = False, header = True)
print('--- GDSC(RNA_Seq, CpNum, mid_dl_input) INTERSECTION [GENES, CELLLINES]: ' + str(tail_cellline_cpnum_df.shape) + '---')
# REMOVE CELLLINES FOR [mid_GDSC2_dl_input]
print('[DL INPUT] REMOVING mid_GDSC2_dl_input OUTER CELL LINES ...')
mid_dl_input_deletion_list = []
for row in mid_dl_input_df.itertuples():
    if row[1] not in common_cellline_list:
        mid_dl_input_deletion_list.append(row[0])
final_dl_input_df = mid_dl_input_df.drop(mid_dl_input_df.index[mid_dl_input_deletion_list]).reset_index(drop = True)
final_dl_input_df.to_csv('./datainfo/mid_data/final_GDSC2_dl_input.txt', index = False, header = True)
print('--- DL FINAL INPUT POINTS: ' + str(final_dl_input_df.shape[0]) + ' ---')



[DL INPUT] REMOVING RNA-Seq/CpNum OUTER CELL LINES ...
--- GDSC(RNA_Seq, CpNum, mid_dl_input) INTERSECTION [GENES, CELLLINES]: (9268, 793)---
[DL INPUT] REMOVING mid_GDSC2_dl_input OUTER CELL LINES ...
--- DL FINAL INPUT POINTS: 38227 ---


In [48]:
pre_cellline_parse()


FILTERING SPARSE FEATURE GENES OF RNA_Seq & CpNum...
(37279, 1049)
(9936, 1049)

FINDING INTERSECTION OF RNA_Seq & CpNum...
--- CONFIRMED ON IDENTICAL OF [RNA_Seq, CpNum] ---
--- GDSC(RNA_Seq, CpNum) INTERSECTION [GENES, CELLLINES]: (9268, 973) ---

[DL INPUT] REMOVING RNA-Seq/CpNum OUTER CELL LINES ...
--- GDSC(RNA_Seq, CpNum, mid_dl_input) INTERSECTION [GENES, CELLLINES]: (9268, 793)---
[DL INPUT] REMOVING mid_GDSC2_dl_input OUTER CELL LINES ...
--- DL FINAL INPUT POINTS: 38227 ---


In [49]:
def pre_gene_parse():
    ParseFile.gene_intersect_gdsc_condense()

In [50]:
gene_pathway_df = pd.read_table('./datainfo/init_data/Selected_Kegg_Pathways2.txt')
gene_list = list(gene_pathway_df['AllGenes'])
intersect_rna_df = pd.read_csv('./datainfo/mid_data/intersect_rnaseq_fpkm_20191101.csv')
intersect_cpnum_df = pd.read_csv('./datainfo/mid_data/intersect_cnv_gistic_20191101.csv')

In [51]:
rna_gene_list = list(intersect_rna_df['symbol'])
rna_gene_set = set(rna_gene_list)
pathway_gene_set = set(gene_list)
common_gene_list = list(rna_gene_set.intersection(pathway_gene_set))

In [52]:
len(common_gene_list)

929

In [53]:
gene_deletion_list = []
for gene in rna_gene_list:
    if gene not in common_gene_list:
        gene_deletion_list.append(gene)
rna_gene_deletion_index = []
for row in intersect_rna_df.itertuples():
    if row[2] in gene_deletion_list:
        rna_gene_deletion_index.append(row[0])
tailed_rna_df = intersect_rna_df.drop(rna_gene_deletion_index).reset_index(drop = True)
tailed_rna_df = tailed_rna_df.fillna(0.0)
tailed_rna_df.to_csv('./datainfo/filtered_data/tailed_rnaseq_fpkm_20191101.csv', index = False, header = True)
# print(tailed_rna_df)
cpnum_gene_deletion_index = []
for row in intersect_cpnum_df.itertuples():
    if row[2] in gene_deletion_list:
        cpnum_gene_deletion_index.append(row[0])
tailed_cpnum_df = intersect_cpnum_df.drop(cpnum_gene_deletion_index).reset_index(drop = True)
tailed_cpnum_df.to_csv('./datainfo/filtered_data/tailed_cnv_gistic_20191101.csv', index = False, header = True)
# print(tailed_cpnum_df)
# print(tailed_rna_df.isnull().sum())
print('--- GDSC(RNA_Seq, CpNum) FINAL [GENES, CELLLINES]: ' + str(tailed_cpnum_df.shape) + ' ---')


--- GDSC(RNA_Seq, CpNum) FINAL [GENES, CELLLINES]: (929, 793) ---


In [54]:
intersect_rna_df[intersect_rna_df['symbol'].isin(common_gene_list)]

,gene_id,symbol,M14,TE-12,PL4,PCI-4B,PCI-30,HSC-39,EMC-BAC-2,DiFi,...,CHL-1,BE2-M17,C2BBe1,C3A,BE-13,Ramos-2G6-4C10,CGTH-W-1,H9,GR-ST,MM1S
61,SIDG00150,ABL1,19.0,16.0,11.0,13.0,43.0,16.0,28.0,27.0,...,16.0,20.0,10.0,14.0,13.0,10.0,24.0,22.0,9.0,5.0
62,SIDG00151,ABL2,13.0,4.0,4.0,11.0,17.0,3.0,5.0,3.0,...,4.0,10.0,5.0,4.0,3.0,4.0,10.0,6.0,6.0,4.0
73,SIDG00167,ACACA,11.0,13.0,22.0,14.0,9.0,21.0,29.0,7.0,...,9.0,18.0,13.0,8.0,13.0,10.0,13.0,15.0,6.0,7.0
101,SIDG00224,ACOX1,5.0,8.0,10.0,5.0,26.0,7.0,14.0,19.0,...,5.0,3.0,9.0,23.0,7.0,3.0,3.0,6.0,6.0,3.0
102,SIDG00226,ACOX3,3.0,2.0,5.0,2.0,3.0,4.0,2.0,5.0,...,2.0,2.0,2.0,3.0,3.0,6.0,3.0,4.0,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,SIDG41439,ZBTB17,6.0,3.0,4.0,4.0,7.0,4.0,5.0,4.0,...,7.0,3.0,3.0,3.0,10.0,7.0,6.0,6.0,14.0,5.0
8895,SIDG41610,ZFYVE16,11.0,5.0,6.0,5.0,3.0,3.0,7.0,2.0,...,16.0,4.0,7.0,6.0,8.0,2.0,5.0,1.0,7.0,2.0
8900,SIDG41607,ZFYVE9,6.0,8.0,2.0,5.0,12.0,5.0,7.0,5.0,...,5.0,7.0,5.0,6.0,4.0,0.0,13.0,4.0,0.0,1.0
8913,SIDG41646,ZMAT3,5.0,2.0,2.0,5.0,6.0,5.0,7.0,2.0,...,4.0,3.0,2.0,9.0,11.0,10.0,7.0,8.0,5.0,0.0


In [55]:
intersect_cpnum_df[intersect_cpnum_df['symbol'].isin(common_gene_list)]

,gene_id,symbol,M14,TE-12,PL4,PCI-4B,PCI-30,HSC-39,EMC-BAC-2,DiFi,...,CHL-1,BE2-M17,C2BBe1,C3A,BE-13,Ramos-2G6-4C10,CGTH-W-1,H9,GR-ST,MM1S
61,SIDG00150,ABL1,0,1,-1,1,1,0,-1,0,...,0,0,-1,-1,-1,-1,0,1,0,0
62,SIDG00151,ABL2,0,0,0,0,1,0,0,0,...,1,1,0,-1,-1,0,1,1,1,1
73,SIDG00167,ACACA,0,0,0,-1,-1,1,1,-1,...,1,0,0,-1,0,0,-1,1,0,0
101,SIDG00224,ACOX1,0,1,0,-1,2,0,1,1,...,1,0,0,1,0,-1,-1,1,0,0
102,SIDG00226,ACOX3,-1,-1,1,-1,-1,0,0,-1,...,-1,0,-1,-1,-1,0,-1,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,SIDG41439,ZBTB17,1,-1,-1,-1,1,0,0,-1,...,1,-1,-1,1,0,0,0,0,0,0
8895,SIDG41610,ZFYVE16,0,-1,1,1,-1,0,0,-1,...,-1,0,1,0,0,0,-1,-1,1,0
8900,SIDG41607,ZFYVE9,1,0,-1,0,1,0,0,-1,...,1,-1,-1,0,0,0,1,0,0,0
8913,SIDG41646,ZMAT3,1,1,1,1,2,1,0,1,...,1,0,0,0,-1,0,1,1,0,2


In [56]:
drug_dict, drug_num_dict, target_dict, target_num_dict = ParseFile.drug_target()
rna_df = pd.read_csv('./datainfo/filtered_data/tailed_rnaseq_fpkm_20191101.csv')
print(rna_df.shape)

(929, 793)


In [57]:
rna_df.head()

,gene_id,symbol,M14,TE-12,PL4,PCI-4B,PCI-30,HSC-39,EMC-BAC-2,DiFi,...,CHL-1,BE2-M17,C2BBe1,C3A,BE-13,Ramos-2G6-4C10,CGTH-W-1,H9,GR-ST,MM1S
0,SIDG00150,ABL1,19.0,16.0,11.0,13.0,43.0,16.0,28.0,27.0,...,16.0,20.0,10.0,14.0,13.0,10.0,24.0,22.0,9.0,5.0
1,SIDG00151,ABL2,13.0,4.0,4.0,11.0,17.0,3.0,5.0,3.0,...,4.0,10.0,5.0,4.0,3.0,4.0,10.0,6.0,6.0,4.0
2,SIDG00167,ACACA,11.0,13.0,22.0,14.0,9.0,21.0,29.0,7.0,...,9.0,18.0,13.0,8.0,13.0,10.0,13.0,15.0,6.0,7.0
3,SIDG00224,ACOX1,5.0,8.0,10.0,5.0,26.0,7.0,14.0,19.0,...,5.0,3.0,9.0,23.0,7.0,3.0,3.0,6.0,6.0,3.0
4,SIDG00226,ACOX3,3.0,2.0,5.0,2.0,3.0,4.0,2.0,5.0,...,2.0,2.0,2.0,3.0,3.0,6.0,3.0,4.0,5.0,3.0


In [58]:
# print(target_dict)
gene_target_num_dict = {}
count = 0
num = 0
for row in rna_df.itertuples():
    if row[2] not in target_dict.keys(): 
        map_index = -1
        count += 1
    else:
        map_index = target_dict[row[2]]
        num += 1
    gene_target_num_dict[row[0]] = map_index
print(count)
print(num)
np.save('./datainfo/filtered_data/gene_target_num_dict.npy', gene_target_num_dict)


596
333


In [59]:
gene_pathway_df = pd.read_table('./datainfo/init_data/Selected_Kegg_Pathways2.txt')
gene_pathway_df = gene_pathway_df.sort_values(by = ['AllGenes']).reset_index(drop = True)
gene_list = list(gene_pathway_df['AllGenes'])
gene_pathway_df = gene_pathway_df.drop(['AllGenes'], axis = 1)
pathway_list = list(gene_pathway_df.columns)

In [60]:
gene_dict = {gene_list[i] : i for i in range(len(gene_list))}

In [61]:
gene_num_dict = {i : gene_list[i] for i in range(len(gene_list))}

In [62]:
pathway_dict = {pathway_list[i] : i for i in range(len(pathway_list))}
pathway_num_dict = {i : pathway_list[i] for i in range(len(pathway_list))}

In [63]:
gene_pathway_matrix = np.zeros((len(gene_list), len(pathway_list))).astype(int)

In [64]:
gene_pathway_matrix.shape

(2144, 46)

In [65]:
gene_pathway_df

,MAPK signaling pathway,ErbB signaling pathway,Ras signaling pathway,Rap1 signaling pathway,Calcium signaling pathway,cGMP-PKG signaling pathway,cAMP signaling pathway,Chemokine signaling pathway,NF-kappa B signaling pathway,HIF-1 signaling pathway,...,GnRH signaling pathway,Estrogen signaling pathway,Prolactin signaling pathway,Thyroid hormone signaling pathway,Adipocytokine signaling pathway,Oxytocin signaling pathway,Glucagon signaling pathway,Relaxin signaling pathway,AGE-RAGE signaling pathway in diabetic complications,Cell cycle
0,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
1,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
2,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
3,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
4,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
2140,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
2141,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test
2142,test,test,test,test,test,test,test,test,test,test,...,test,test,test,test,test,test,test,test,test,test


In [66]:
for gene_row in gene_pathway_df.itertuples():
    pathway_index = 0
    for gene in gene_row[1:]:
        if gene != 'test':
            gene_pathway_matrix[gene_dict[gene], pathway_index] = 1.0
        pathway_index += 1

In [67]:
gene_pathway_matrix

array([[0, 1, 1, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [70]:
np.save('./datainfo/mid_data/gene_pathway_matrix.npy', gene_pathway_matrix)

In [71]:
order_gene_pathway_df = pd.DataFrame(data = gene_pathway_matrix,
            index = [gene for gene in gene_list],
            columns = [pathway for pathway in pathway_list])

In [72]:
order_gene_pathway_df.to_csv('./datainfo/mid_data/Ordered_Selected_Kegg_Pathways2.csv', index = True, header = True)


In [73]:
order_gene_pathway_df = pd.read_csv('./datainfo/mid_data/Ordered_Selected_Kegg_Pathways2.csv')
rna_df = pd.read_csv('./datainfo/filtered_data/tailed_rnaseq_fpkm_20191101.csv')
cpnum_df = pd.read_csv('./datainfo/filtered_data/tailed_cnv_gistic_20191101.csv')
rna_gene_list = list(rna_df['symbol'])
deletion_list = []

In [74]:
order_gene_pathway_df

,Unnamed: 0,MAPK signaling pathway,ErbB signaling pathway,Ras signaling pathway,Rap1 signaling pathway,Calcium signaling pathway,cGMP-PKG signaling pathway,cAMP signaling pathway,Chemokine signaling pathway,NF-kappa B signaling pathway,...,GnRH signaling pathway,Estrogen signaling pathway,Prolactin signaling pathway,Thyroid hormone signaling pathway,Adipocytokine signaling pathway,Oxytocin signaling pathway,Glucagon signaling pathway,Relaxin signaling pathway,AGE-RAGE signaling pathway in diabetic complications,Cell cycle
0,ABL1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,ABL2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACACA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,ACACB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,ACOX1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,ZFYVE9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2140,ZMAT3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2141,ZNF274,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2142,dest,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
for row in order_gene_pathway_df.itertuples():
    if row[1] not in rna_gene_list:
        deletion_list.append(row[0])

In [76]:
tailed_gene_pathway_df = order_gene_pathway_df.drop(deletion_list).reset_index(drop = True)
tailed_gene_pathway_df.to_csv('./datainfo/filtered_data/Tailed_Selected_Kegg_Pathways2.csv', index = False, header = True)
tailed_gene_pathway_df = tailed_gene_pathway_df.drop(columns = ['Unnamed: 0'])

In [77]:
tailed_gene_pathway_df

,MAPK signaling pathway,ErbB signaling pathway,Ras signaling pathway,Rap1 signaling pathway,Calcium signaling pathway,cGMP-PKG signaling pathway,cAMP signaling pathway,Chemokine signaling pathway,NF-kappa B signaling pathway,HIF-1 signaling pathway,...,GnRH signaling pathway,Estrogen signaling pathway,Prolactin signaling pathway,Thyroid hormone signaling pathway,Adipocytokine signaling pathway,Oxytocin signaling pathway,Glucagon signaling pathway,Relaxin signaling pathway,AGE-RAGE signaling pathway in diabetic complications,Cell cycle
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
925,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
927,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
tailed_rna_cellline_list = list(rna_df.columns)
tailed_rna_gene_list = list(rna_df['symbol'])
tailed_cpnum_cellline_list = list(cpnum_df.columns)
tailed_cpnum_gene_list = list(cpnum_df['symbol']) 
error = 0
for (rna_cl, cpnum_cl) in zip(tailed_rna_cellline_list, tailed_cpnum_cellline_list):
    if rna_cl != cpnum_cl: error = 1
for (rna_gene, cpnum_gene) in zip(tailed_rna_gene_list, tailed_cpnum_gene_list):
    if rna_gene != cpnum_gene: error = 2
if error == 0: 
    print('--- CONFIRMED ON IDENTICAL OF [RNA_Seq, CpNum] ---')
np.save('./datainfo/filtered_data/gene_pathway_matrix.npy', tailed_gene_pathway_df.values)

--- CONFIRMED ON IDENTICAL OF [RNA_Seq, CpNum] ---


In [79]:
dir_opt = '/datainfo'
deletion_list = []
final_dl_input_df = pd.read_table('./datainfo/mid_data/final_GDSC2_dl_input.txt', delimiter = ',')
drug_map_dict, drug_dict, gene_target_num_dict = LoadData(dir_opt).pre_load_dict()
target_index_list = gene_target_num_dict.values()
drug_target_matrix = np.load('./datainfo/filtered_data/drug_target_matrix.npy')

In [82]:
drug_map_dict[2]

KeyError: 2

In [83]:
for row in final_dl_input_df.itertuples():
    drug_a = drug_map_dict[row[2]]
    cellline_name = row[1]
    # DRUG_A AND 929 TARGET GENES
    drug_a_target_list = []
    drug_index = drug_dict[drug_a]
    for target_index in target_index_list:
        if target_index == -1 : 
            effect = 0
        else:
            effect = drug_target_matrix[drug_index, target_index]
        drug_a_target_list.append(effect)
    if all([a == 0 for a in drug_a_target_list]): 
        deletion_list.append(row[0])
print('=====================' + str(len(deletion_list)))

=====================21466


In [84]:
zero_final_dl_input_df = final_dl_input_df.drop(final_dl_input_df.index[deletion_list]).reset_index(drop = True)
zero_final_dl_input_df.to_csv('.' + dir_opt + '/filtered_data/zerofinal_GDSC2_dl_input.txt', index = False, header = True)
print(zero_final_dl_input_df)

      CELL_LINE_NAME    DRUG_NAME       AUC
0              22RV1     Afatinib  0.879934
1              22RV1   Crizotinib  0.973519
2              22RV1   Dabrafenib  0.980118
3              22RV1    Daporinad  0.765028
4              22RV1    Dasatinib  0.975473
...              ...          ...       ...
16756          huH-1    Tamoxifen  0.942290
16757          huH-1   Trametinib  0.925965
16758          huH-1  Ulixertinib  0.938272
16759          huH-1  Vinblastine  0.834914
16760          huH-1   Vorinostat  0.811646

[16761 rows x 3 columns]


In [85]:
def k_fold_split (random_mode, k, place_num):
    dir_opt = '/datainfo2'
    if random_mode == True:
        ParseFile.input_random_condense()
    ParseFile.split_k_fold(k, place_num)

k_fold_split(False, 5, 1)


      CELL_LINE_NAME      DRUG_NAME       AUC
0         MDA-MB-436      Tamoxifen  0.966082
1             EFO-21    Leflunomide  0.903636
2            OVCAR-4     Dabrafenib  0.983033
3             PWR-1E       Afatinib  0.888368
4            HCT-116     Linsitinib  0.959935
...              ...            ...       ...
16756          OVISE      Lapatinib  0.914364
16757          8305C    Ulixertinib  0.937430
16758        IGROV-1       Afatinib  0.603225
16759      NCI-H1651        MK-1775  0.671480
16760          ST486  Staurosporine  0.238394

[16761 rows x 3 columns]

--------TRAIN-TEST SPLIT WITH TEST FROM 0 TO 3352--------
      CELL_LINE_NAME      DRUG_NAME       AUC
3352           HSC-3    Palbociclib  0.903366
3353      LCLC-97TM1      Lapatinib  0.948285
3354       RPMI-8866  Staurosporine  0.293089
3355         Hs-746T     Crizotinib  0.713783
3356              HH      Tamoxifen  0.950498
...              ...            ...       ...
16756          OVISE      Lapatinib  0.91

In [86]:
# DOING K-FOLD VALIDATION IN 100% DATASET
random_mode = False
k = 5
place_num = 1
k_fold_split(random_mode, k, place_num)

      CELL_LINE_NAME      DRUG_NAME       AUC
0         MDA-MB-436      Tamoxifen  0.966082
1             EFO-21    Leflunomide  0.903636
2            OVCAR-4     Dabrafenib  0.983033
3             PWR-1E       Afatinib  0.888368
4            HCT-116     Linsitinib  0.959935
...              ...            ...       ...
16756          OVISE      Lapatinib  0.914364
16757          8305C    Ulixertinib  0.937430
16758        IGROV-1       Afatinib  0.603225
16759      NCI-H1651        MK-1775  0.671480
16760          ST486  Staurosporine  0.238394

[16761 rows x 3 columns]

--------TRAIN-TEST SPLIT WITH TEST FROM 0 TO 3352--------
      CELL_LINE_NAME      DRUG_NAME       AUC
3352           HSC-3    Palbociclib  0.903366
3353      LCLC-97TM1      Lapatinib  0.948285
3354       RPMI-8866  Staurosporine  0.293089
3355         Hs-746T     Crizotinib  0.713783
3356              HH      Tamoxifen  0.950498
...              ...            ...       ...
16756          OVISE      Lapatinib  0.91

# Data loading

In [87]:
!python load_data.py

LOADING ALL DATA...
-----0 to 256-----
(256, 2787)
(256, 1)
-----256 to 512-----
(256, 2787)
(256, 1)
-----512 to 768-----
(256, 2787)
(256, 1)
-----768 to 1024-----
(256, 2787)
(256, 1)
-----1024 to 1280-----
(256, 2787)
(256, 1)
-----1280 to 1536-----
(256, 2787)
(256, 1)
-----1536 to 1792-----
(256, 2787)
(256, 1)
-----1792 to 2048-----
(256, 2787)
(256, 1)
-----2048 to 2304-----
(256, 2787)
(256, 1)
-----2304 to 2560-----
(256, 2787)
(256, 1)
-----2560 to 2816-----
(256, 2787)
(256, 1)
-----2816 to 3072-----
(256, 2787)
(256, 1)
-----3072 to 3328-----
(256, 2787)
(256, 1)
-----3328 to 3584-----
(256, 2787)
(256, 1)
-----3584 to 3840-----
(256, 2787)
(256, 1)
-----3840 to 4096-----
(256, 2787)
(256, 1)
-----4096 to 4352-----
(256, 2787)
(256, 1)
-----4352 to 4608-----
(256, 2787)
(256, 1)
-----4608 to 4864-----
(256, 2787)
(256, 1)
-----4864 to 5120-----
(256, 2787)
(256, 1)
-----5120 to 5376-----
(256, 2787)
(256, 1)
-----5376 to 5632-----
(256, 2787)
(256, 1)
-----5632 to 5888----

In [88]:
from load_data import *

dir_opt = '/datainfo'
post_data_path = '.' + dir_opt + '/post_data'
if os.path.exists(post_data_path) == False:
        os.mkdir(post_data_path)
batch_size = 256

In [89]:
X_train, y_train, X_test, y_test, X_features, y_target = LoadData(dir_opt).load_all(batch_size,post_data_path) # train data, test data, x,y는 전체 데이터

LOADING ALL DATA...
-----0 to 256-----
(256, 2787)
(256, 1)
-----256 to 512-----
(256, 2787)
(256, 1)
-----512 to 768-----
(256, 2787)
(256, 1)
-----768 to 1024-----
(256, 2787)
(256, 1)
-----1024 to 1280-----
(256, 2787)
(256, 1)
-----1280 to 1536-----
(256, 2787)
(256, 1)
-----1536 to 1792-----
(256, 2787)
(256, 1)
-----1792 to 2048-----
(256, 2787)
(256, 1)
-----2048 to 2304-----
(256, 2787)
(256, 1)
-----2304 to 2560-----
(256, 2787)
(256, 1)
-----2560 to 2816-----
(256, 2787)
(256, 1)
-----2816 to 3072-----
(256, 2787)
(256, 1)
-----3072 to 3328-----
(256, 2787)
(256, 1)
-----3328 to 3584-----
(256, 2787)
(256, 1)
-----3584 to 3840-----
(256, 2787)
(256, 1)
-----3840 to 4096-----
(256, 2787)
(256, 1)
-----4096 to 4352-----
(256, 2787)
(256, 1)
-----4352 to 4608-----
(256, 2787)
(256, 1)
-----4608 to 4864-----
(256, 2787)
(256, 1)
-----4864 to 5120-----
(256, 2787)
(256, 1)
-----5120 to 5376-----
(256, 2787)
(256, 1)
-----5376 to 5632-----
(256, 2787)
(256, 1)
-----5632 to 5888----

In [90]:
X_train.shape # 2787 = 929 (Genes) * 3(features:rnaseq, cnv, drug_target_information)

(13409, 2787)

In [91]:
X_train.shape # 2787 = 929 (Genes) * 3(features:rnaseq, cnv, drug_target_information)

(13409, 2787)

In [92]:
X_train

array([[14.,  1.,  0., ...,  3.,  0.,  0.],
       [20., -1.,  0., ...,  4., -1.,  0.],
       [14.,  0.,  0., ...,  2.,  0.,  0.],
       ...,
       [23.,  0.,  0., ...,  3.,  0.,  0.],
       [20., -1.,  0., ...,  5.,  0.,  0.],
       [ 9.,  0.,  0., ...,  3.,  0.,  0.]])

In [93]:
for i in range(0, len(X_train[1]),3):
    #print(i)
    print(X_train[1][i:i+3])

[20. -1.  0.]
[6. 0. 0.]
[12. -1.  0.]
[3. 0. 0.]
[3. 0. 0.]
[14.  1.  0.]
[18.  0.  0.]
[690.   1.   0.]
[863.   0.   0.]
[6. 0. 0.]
[11.  1.  0.]
[1. 0. 0.]
[16.  0.  0.]
[16.  0.  0.]
[7. 1. 0.]
[ 6. -1.  0.]
[2. 0. 0.]
[42.  0.  0.]
[24.  1.  0.]
[10. -1.  0.]
[13. -1.  0.]
[ 1. -1.  0.]
[3. 0. 0.]
[17.  0.  0.]
[17.  2.  0.]
[ 6. -1.  0.]
[ 7. -1.  0.]
[0. 0. 0.]
[132.   0.   0.]
[ 1. -1.  0.]
[5. 0. 0.]
[3. 0. 0.]
[18.  0.  0.]
[21.  1.  0.]
[10. -1.  0.]
[5. 0. 0.]
[12.  1.  0.]
[12.  1.  0.]
[2. 1. 0.]
[2. 0. 0.]
[114.   2.   0.]
[ 0. -1.  0.]
[150.   0.   0.]
[55.  0.  0.]
[48.  0.  0.]
[57.  1.  0.]
[8. 0. 0.]
[7. 2. 0.]
[5. 0. 0.]
[ 1. -1.  0.]
[7. 0. 0.]
[248.  -1.   0.]
[34. -1.  0.]
[6. 0. 0.]
[10.  0.  0.]
[ 7. -1.  0.]
[ 1. -1.  0.]
[68. -1.  0.]
[44.  0.  0.]
[37.  1.  0.]
[50.  1.  0.]
[11.  1.  0.]
[14.  0.  0.]
[16.  1.  0.]
[ 7. -1.  0.]
[17.  1.  0.]
[10.  0.  0.]
[19. -1.  0.]
[2. 0. 0.]
[106.   1.   0.]
[76.  0.  0.]
[12.  2.  0.]
[6. 1. 0.]
[12. -1.  0.]
[4. 0.

In [94]:
y_train # AUC value

array([[0.903366],
       [0.948285],
       [0.293089],
       ...,
       [0.603225],
       [0.67148 ],
       [0.238394]])

In [95]:
train_input_df, input_num, num_feature, rna_df, cpnum_df, num_gene, num_pathway = LoadData(dir_opt).pre_load_train()

In [96]:
train_input_df.iloc[0:3]

,CELL_LINE_NAME,DRUG_NAME,AUC
0,HSC-3,Palbociclib,0.903366
1,LCLC-97TM1,Lapatinib,0.948285
2,RPMI-8866,Staurosporine,0.293089
